# Canopy simulation tutorial

This notebook contains sample cases for the Eradiate preview session of Dec 7th 2021. In this tutorial, we introduce the canopy simulation capabilities of Eradiate. We will explore the features and interfaces of different objects used to construct a scene, simulate radiative transfer and visualise results.

In [ ]:
# We load the Rich notebook extension for improved object inspection
%load_ext rich

# We import general processing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# We import the top-level Eradiate module
import eradiate

# For convenience, we alias a few components
from eradiate import unit_registry as ureg
import eradiate.scenes as ertsc
import eradiate.experiments as ertxp

# We'll use the monochromatic mode in double precision
eradiate.set_mode("mono_double")

# We activate a few convenient presets
eradiate.notebook.install()
eradiate.plot.set_style(rc={"figure.dpi": 96})
eradiate.config.progress = 1  # Restrict progress display to the highest level

We start with the definition of a couple of utility functions which will simplify the visualisation of results.

In [ ]:
def show_camera(exp, measure_id):
    """
    Display the output of a monochromatic camera measure.
    """
    fig, ax = plt.subplots(1,1)
    exp.results[measure_id]["radiance"].squeeze(drop=True).plot.imshow(
        ax=ax, origin="upper", robust=True, cmap="Greys_r", add_colorbar=False
    )
    ax.set_aspect(1)
    plt.show()
    plt.close()

def show_brf(exp, measure_id):
    """
    Display the BRF output of a distant radiance measure.
    """
    fig, ax = plt.subplots(1,1)
    with plt.style.context({"lines.linestyle": ":", "lines.marker": "."}):
        exp.results[measure_id]["brf"].squeeze(drop=True).plot(ax=ax, x="vza")
    plt.show()
    plt.close()

## Rendering a surface


First, let's create a very simple scene consisting of a default Lambertian surface observed by a camera. We start by defining the surface and set its width to 10 m (the default is much larger).

In [ ]:
lambertian_surface=ertsc.surface.LambertianSurface(
    width=10 * ureg.m, reflectance=0.5
)

Now, let's make a monochromatic render of this scene. We'll produce a 4:3 ratio image with a fairly low resolution sample count—all we want is to visualise the scene.

Note that at the moment, Eradiate's rendering capabilities are limited to the monochromatic mode.

In [ ]:
camera_oblique = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=512,
)
exp = ertxp.RamiExperiment(
    surface=lambertian_surface, measures=camera_oblique
)

In [ ]:
exp.run()
show_camera(exp, "camera_oblique")

You might notice rendering artifacts. This is because Eradiate's camera settings are currently not optimised for rendering. This will be improved in future releases.

## Rendering a surface and canopy


Let's add a canopy above this ground patch. We'll configure a homogeneous canopy composed of randomly oriented floating disks. This is done using the [`DiscreteCanopy` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.biosphere.DiscreteCanopy.html), or, rather, its [`homogeneous()`](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.biosphere.DiscreteCanopy.html) class constructor. We will use abstract parameters for the sake of demonstrating this feature:

In [ ]:
homogeneous_canopy = eradiate.scenes.biosphere.DiscreteCanopy.homogeneous(
    l_vertical=1.0 * ureg.m,
    l_horizontal=10.0 * ureg.m,
    lai=2.0,
    leaf_radius=10 * ureg.cm,
)

We add this canopy to a scene with a surface using the [`RamiExperiment` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.experiments.RamiExperiment.html). Note that this time, we don't set the surface size: it is automatically adjusted to match the extents we set for the canopy object.

In [ ]:
exp = ertxp.RamiExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    measures=camera_oblique,
)
exp.run()
show_camera(exp, "camera_oblique")


## Rendering a surface and padded canopy


In order to simulate the fact that our canopy is periodic, we can pad it with clones of itself. Let's start with just of few of them.

**Tip**: If the render time is too long, you can reduce the number of radiance samples per image pixel by adjusting the measure's `spp` parameter.


In [ ]:
camera_oblique = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)

exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=1,
    measures=camera_oblique,
)
exp.run()
show_camera(exp, "camera_oblique")

Now, let's relocate our camera to the top of canopy. We will observe it at a grazing angle.

In [ ]:
camera_toc = eradiate.scenes.measure.PerspectiveCameraMeasure(
    id="camera_toc",
    origin=[15, 15, 2] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)
exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[camera_toc],
)
exp.run()
show_camera(exp, "camera_toc")

## Computing the BRF in the principal plane

Now that we know how to build our scene, we are going to compute the BRF in the principal plane for a similar scene. This is similar to the test cases for which we can submit results to the RAMI online checker (ROMC.)

In order to observe something meaningful, let's set the Sun zenith angle to 30°.

In [ ]:
illumination = eradiate.scenes.illumination.DirectionalIllumination(
    zenith=30.0, azimuth=0.0
)

brf_pplane = eradiate.scenes.measure.MultiDistantMeasure.from_viewing_angles(
    id="brf_pplane",
    azimuths=0.0,
    zeniths=np.arange(-75, 76, 5),
    spp=50000,
)

exp = eradiate.experiments.RamiExperiment(
    illumination=illumination,
    surface=eradiate.scenes.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    padding=10,
    measures=[brf_pplane],
)
exp.run()
show_brf(exp, "brf_pplane")

There still is some Monte Carlo noise, but we're keeping the number of samples low in order so that the simulation remains short.

**Exercises for the interested reader**

1. Change the viewing zenith angle and verify that the hot spot moves accordingly.
2. Change the viewing azimuth angle and adjust the measure so as to keep it aligned with the principal plane.
3. Compute the BRF in a plane orthgonal to the principal plane.
4. Gradually increase padding and determine from which value the quasi-periodic approximation can be considered valid.

## Computing the BRF in the principal plane with an atmosphere

Finally, we can combine our canopy with an atmosphere. This uses a different experiment class ([`Rami4ATMExperiment`](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.experiments.Rami4ATMExperiment.html)).

In [ ]:
eradiate.set_mode("mono_double")

Rayleigh atmosphere only, mono mode

* TOC view, show haze (requires medium ID update)
* TOA view, show haze
* TOA BRF (ex: show hot spot broadening vs wavelength)

Activate absorption, ckd mode

* TOA BRF, full band computation

In [ ]:
camera_toc = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_toc",
    origin=[15, 15, 2] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)
exp = eradiate.experiments.Rami4ATMExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    atmosphere=ertsc.atmosphere.MolecularAtmosphere(has_absorption=False),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[camera_toc],
)
exp.run()
show_camera(exp, "camera_toc")